In [3]:
import os
import re
import sys
import tqdm
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

import random
import seaborn as sns

# Libraries required for plotting field
import matplotlib.patches as patches
from matplotlib.patches import Arc
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import animation
from IPython.display import HTML
from IPython import display


# Custom libraries
sys.path.append('../')
from src.utils.feature_extraction import extract_play_outcome_features, extract_formation_features, extract_foul_features, extract_injury_features, extract_game_features
from definitions.fouls_def import ignore_fouls
from src.utils.viz_functions import drawPitch, drawPocket, animatePlay_Generic, animateScores
from src.utils.play_preprocessing import extractPlay, preprocessPlay_refBallInit, preprocessPlay_refQB, preprocessPlay_refQB_NFrames
from src.utils.player_influence import extract_play_players_influence, gaussian_player_influence_score
from src.utils.field_price_functions import calculate_field_price, gaussian_field_price
from src.utils.calculate_score import calculate_score

In [4]:
plays = pd.read_csv("../input/plays.csv")


In [17]:
random.seed(1225)

sample_plays = \
  (plays
   .query("dropBackType in ['TRADITIONAL', 'SCRAMBLE'] & passResult in ['C', 'I', 'S', 'R'] ")
   .groupby(['dropBackType', 'passResult'])
   .sample(n=2, replace=True)
   .reset_index()
   )

In [18]:
full_df = pd.DataFrame()

for file_number in range(1,9):

    pos_data = \
        (pd.read_csv(f"../input/week{file_number}.csv")
        .set_index(["gameId", "playId"]) 
        .join(sample_plays.set_index(["gameId", "playId"]),
            how="inner")
        ).reset_index()
    pos_data['weekId'] = f"week{file_number}.csv"

    full_df=pd.concat([full_df, pos_data])



In [19]:
(full_df[['weekId', 'gameId', 'playId']]
  .groupby(['weekId', 'gameId', 'playId'])
  .size()
  .reset_index()
  .drop(0, axis=1)
  .to_csv("../input/shorter_plays_for_analysis.csv", index=False))